In [1]:
import numpy as np
from scipy import linalg as la
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%run camera_functions
%matplotlib inline

In [2]:
def compute_view_matrix(position, target, up):
    '''
    Inputs: 
        position : 3D coordinates of the camera center in world coordinates
        target : 3D coordinates of the camera target in world coordinates
        up : vector specifying the up direction of the camera. should be orthogonal to (target-position)
    Outputs:
        P : 3 x 4 camera extrinsic matrix, a concatenation of rotation matrix and translation matrix
    '''
    L = target - position
    L = L / np.linalg.norm(L)
    s = np.cross(L, up)
    s = s / np.linalg.norm(s)
    u_prime = np.cross(s,L)
    R = np.r_[s,u_prime,-L].reshape((3,3))
    T = -R @ position
    P = np.c_[R,T]
    return P

def project(position,target,up,X):
    '''
    Given camera extrinsic details, and 3D points X, 
    return the homogenous coordinates of Y,
    which are projected into the camera plane.
    See compute_view_matrix() details
    '''
    P = compute_view_matrix(position,target,up)
    if len(X.shape) > 1:
        _,n = X.shape
        Y = P @ np.r_[X,np.ones((1,n))]
        Y /= Y[2,:]
        return Y
    else:
        Y = P @ np.r_[X,1]
        Y /= Y[2]
        return Y

def fundamental_matrix(Y,Yp):
    '''
    Inputs:
        Y : all keypoints from camera 1 in homogenous coordinates in a (3 x n_kp) matrix
        Yp : same from camera 2
    Outputs:
        F : fundamental matrix between stereo cameras
    Desc:
        Solved using the 7-point algorithm. It is assumed that matching keypoints are aligned
        by column. At least 7 matches required. 
    TODO: 
        Implement normalized version. Camera intrinsics needed?
    '''
    n = Y.shape[1]
    X = np.zeros((n,9))
    for i in range(n):
        X[i] = np.outer(Yp[:,i],Y[:,i]).flatten()
        
    U,S,Vh = la.svd(X)
    F = Vh[len(Vh)-1].reshape((3,3))
    U,S,Vh = la.svd(F)
    S[2] = 0
    F = U @ np.diag(S) @ Vh
    return F

In [3]:
def rigid_normalize(x,e):
    # Transform x to be at the origin and compute L
    L = np.eye(3)
    L[:2,2] = -x[:2]
#     e = L @ e
    # Find rotation theta by using the epipolar line
    theta = np.arctan( - (e[1] - e[2] * x[1]) / (e[0] - e[2] * x[0]) )
    R = np.array( [ [np.cos(theta), -np.sin(theta),0],[np.sin(theta),np.cos(theta),0],[0,0,1] ] )
    T = R @ L
    # Enforce that RLe[0] = 1
#     T /= (T @ e)[0]
#     print( (T @ e)[2])
    print( (T @ e)[2] / (T @ e)[0])
    return T
    

In [4]:
def fundamental_normalize(x,xp,e,ep,F0):
    '''
    Calculate the fundamental matrix between a pair of points that are
    normalized to be at the origin, and for the epipoles normalized to
    be on the x axis
    '''
    print('f = ')
    T = rigid_normalize(x,e)
    print('f\' = ')
    Tp = rigid_normalize(xp,ep)
#     F = Tp @ F0 @ la.pinv(T)
    F = la.pinv(Tp.T) @ F0 @ la.pinv(T)
    return T,Tp,F
    

In [14]:
X = np.zeros(3)
pos1 = np.array([0,1,0])
tgt1 = np.array([0,0,0])
up1 = np.array([0,0,1])
pos2 = np.array([1,0,0])
tgt2 = np.array([0,0,0])
up2 = np.array([0,0,1])

Y1 = project(pos1,tgt1,up1,X)
Y2 = project(pos2,tgt2,up2,X)
e1 = project(pos1,tgt1,up1,pos2)
e2 = project(pos2,tgt2,up2,pos1)
F0 = np.array([[0,-.5,0],[-.5,0,.5],[0,-.5,0]])



In [19]:
T,Tp,F = fundamental_normalize(Y1,Y2,e1,e2,F0)

f = 
1.0
f' = 
-1.0


In [20]:
print(F)

[[ 0.  -0.5  0. ]
 [-0.5  0.   0.5]
 [ 0.  -0.5  0. ]]
